## Exploring the school index info to generate historical and official win-loss percentage data.

### Data Sources

1. [Sports Reference CFB School Index](https://www.sports-reference.com/cfb/schools/)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

Import the school index data that has the original win-loss-tie data and percentages, along with information about the official NCAA record after considering forfeits and vacated games.

In [2]:
data = pd.read_csv('Data/school_index_sr.csv')

Some general info about the dataset.

In [3]:
print(data.shape)
print(data.isnull().sum())
data.head()

(298, 20)
Rk          0
School      0
From        0
To          0
Yrs         0
G           0
W           0
L           0
T           0
Pct         0
G.1         0
W.1         0
L.1         0
T.1         0
Pct.1     135
SRS         0
SOS         0
AP          0
CC          0
Notes     231
dtype: int64


,Rk,School,From,To,Yrs,G,W,L,T,Pct,G.1,W.1,L.1,T.1,Pct.1,SRS,SOS,AP,CC,Notes
0,1,Air Force,1957,2019,63,705,384,325,12,0.541,26,12,13,1,0.481,1.02,-0.57,6,3,NaN
1,2,Akron,1975,2019,34,383,150,230,3,0.396,3,1,2,0,0.500,-11.46,-7.08,0,1,NaN
2,3,Alabama,1902,2019,116,1213,919,303,42,0.744,70,41,26,3,0.596,13.44,4.15,56,26,record adjusted to 890-312-41 by NCAA
3,4,Alameda Coast Guard,1943,1944,2,15,8,4,3,0.633,0,0,0,0,NaN,-0.16,-5.13,0,0,NaN
4,5,Alcorn State,1977,1977,1,11,3,8,0,0.273,0,0,0,0,NaN,-17.10,-9.19,0,0,NaN


Two things stand out to us immediately.  First, some of these schools are only represented from past seasons, but are not currently competing.  See the feature column `'To'`.  We will filter our data down to keep only teams competing in the updcoming season.  The other thing to note is that the information in the Win-Loss-Tie columns, along with the win percentages are not all officially accurate.  The NCAA sometimes, after reviewing a teams eligibality, decides that a teams wins must be [forfieted or vacated](https://www.sports-reference.com/cfb/friv/forfeits.html).  So, officially, there is an adjusted official win percentage, while the one displayed in the column labeled `'Pct'` is more of an "on the day of" win percentage.

In what follows we will first remove the teams that are not currently competing.  Then we will create several new features for displaying the official win-loss-tie and win percentage data, taking into account the information in the `'Notes'` feature.

In [4]:
# defining the stuff_to_keep dataset that we will transform below.
stuff_to_keep = data[data['To'] == 2019]

General information about the `'stuff_to_keep'` dataframe.

In [5]:
print(stuff_to_keep.shape)
print(stuff_to_keep.isnull().sum())
stuff_to_keep.head()

(130, 20)
Rk         0
School     0
From       0
To         0
Yrs        0
G          0
W          0
L          0
T          0
Pct        0
G.1        0
W.1        0
L.1        0
T.1        0
Pct.1      5
SRS        0
SOS        0
AP         0
CC         0
Notes     67
dtype: int64


,Rk,School,From,To,Yrs,G,W,L,T,Pct,G.1,W.1,L.1,T.1,Pct.1,SRS,SOS,AP,CC,Notes
0,1,Air Force,1957,2019,63,705,384,325,12,0.541,26,12,13,1,0.481,1.02,-0.57,6,3,NaN
1,2,Akron,1975,2019,34,383,150,230,3,0.396,3,1,2,0,0.500,-11.46,-7.08,0,1,NaN
2,3,Alabama,1902,2019,116,1213,919,303,42,0.744,70,41,26,3,0.596,13.44,4.15,56,26,record adjusted to 890-312-41 by NCAA
7,8,Appalachian State,1972,2019,16,174,97,72,5,0.572,4,4,0,0,1.000,-3.06,-6.96,0,1,NaN
8,9,Arizona,1931,2019,86,920,486,422,23,0.534,20,9,10,1,0.500,1.72,0.59,8,6,NaN


Notice that there are 67 nulls in the `'Notes'` feature.  Let's explore the different information that is housed in this column.

In [6]:
# first, extend the number of displayed rows
pd.options.display.max_rows = 200

In [7]:
# now, reset the index
stuff_to_keep.reset_index(inplace=True)
stuff_to_keep.head()

,index,Rk,School,From,To,Yrs,G,W,L,T,...,G.1,W.1,L.1,T.1,Pct.1,SRS,SOS,AP,CC,Notes
0,0,1,Air Force,1957,2019,63,705,384,325,12,...,26,12,13,1,0.481,1.02,-0.57,6,3,NaN
1,1,2,Akron,1975,2019,34,383,150,230,3,...,3,1,2,0,0.500,-11.46,-7.08,0,1,NaN
2,2,3,Alabama,1902,2019,116,1213,919,303,42,...,70,41,26,3,0.596,13.44,4.15,56,26,record adjusted to 890-312-41 by NCAA
3,7,8,Appalachian State,1972,2019,16,174,97,72,5,...,4,4,0,0,1.000,-3.06,-6.96,0,1,NaN
4,8,9,Arizona,1931,2019,86,920,486,422,23,...,20,9,10,1,0.500,1.72,0.59,8,6,NaN


Now, drop the unneeded `'index'` column.

In [8]:
stuff_to_keep.drop(columns='index', axis=1, inplace=True)
stuff_to_keep.head()

/Users/blakewallace/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,Rk,School,From,To,Yrs,G,W,L,T,Pct,G.1,W.1,L.1,T.1,Pct.1,SRS,SOS,AP,CC,Notes
0,1,Air Force,1957,2019,63,705,384,325,12,0.541,26,12,13,1,0.481,1.02,-0.57,6,3,NaN
1,2,Akron,1975,2019,34,383,150,230,3,0.396,3,1,2,0,0.500,-11.46,-7.08,0,1,NaN
2,3,Alabama,1902,2019,116,1213,919,303,42,0.744,70,41,26,3,0.596,13.44,4.15,56,26,record adjusted to 890-312-41 by NCAA
3,8,Appalachian State,1972,2019,16,174,97,72,5,0.572,4,4,0,0,1.000,-3.06,-6.96,0,1,NaN
4,9,Arizona,1931,2019,86,920,486,422,23,0.534,20,9,10,1,0.500,1.72,0.59,8,6,NaN


Next, we explore the data in the `'Notes'` feature.

In [9]:
stuff_to_keep[stuff_to_keep['Notes'].notnull()]['Notes']

2      record adjusted to 890-312-41 by NCAA
6      record adjusted to 682-480-37 by NCAA
7       record adjusted to 183-236-2 by NCAA
9      record adjusted to 744-429-42 by NCAA
11     record adjusted to 584-565-40 by NCAA
17     record adjusted to 568-512-31 by NCAA
24     record adjusted to 516-555-31 by NCAA
27      record adjusted to 321-295-3 by NCAA
32     record adjusted to 508-235-16 by NCAA
33      record adjusted to 351-250-4 by NCAA
37     record adjusted to 728-481-40 by NCAA
39     record adjusted to 429-346-14 by NCAA
41     record adjusted to 460-655-38 by NCAA
42     record adjusted to 620-529-34 by NCAA
43     record adjusted to 512-641-45 by NCAA
45     record adjusted to 472-600-34 by NCAA
47     record adjusted to 492-551-36 by NCAA
49      record adjusted to 205-295-5 by NCAA
50      record adjusted to 230-210-7 by NCAA
52      record adjusted to 351-296-7 by NCAA
53     record adjusted to 769-400-47 by NCAA
57      record adjusted to 315-337-8 by NCAA
59     rec

Visual inspection of this data reveals that every non-empty note contains an officially adjusted win-loss-tie record, and nothing else.

Here is a look at how the win percentage is calculated.  Note that each tie is counted as a half win.

In [42]:
(2*919+42)/(2*(919+303+42))

0.7436708860759493

In [43]:
(2*890+41)/(2*(890+312+41))

0.7325020112630732

Now, we create a dictionary to hold the index and the official record.

In [10]:
dict_index_record = {}

for index, value in stuff_to_keep['Notes'].items():
    if type(value) == str:
#         print(index, value[19:29])
        dict_index_record[index] = value[19:29].split('-')
dict_index_record
#     win_loss_tie = value[19:29].split('-')
#     dict_win_loss_tie[index] = 
#     print(win_loss_tie)

{2: ['890', '312', '41'],
 6: ['682', '480', '37'],
 7: ['183', '236', '2 '],
 9: ['744', '429', '42'],
 11: ['584', '565', '40'],
 17: ['568', '512', '31'],
 24: ['516', '555', '31'],
 27: ['321', '295', '3 '],
 32: ['508', '235', '16'],
 33: ['351', '250', '4 '],
 37: ['728', '481', '40'],
 39: ['429', '346', '14'],
 41: ['460', '655', '38'],
 42: ['620', '529', '34'],
 43: ['512', '641', '45'],
 45: ['472', '600', '34'],
 47: ['492', '551', '36'],
 49: ['205', '295', '5 '],
 50: ['230', '210', '7 '],
 52: ['351', '296', '7 '],
 53: ['769', '400', '47'],
 57: ['315', '337', '8 '],
 59: ['351', '277', '15'],
 61: ['573', '382', '30'],
 64: ['555', '566', '36'],
 65: ['631', '529', '50'],
 67: ['863', '363', '37'],
 69: ['161', '314', '3 '],
 71: ['315', '578', '14'],
 72: ['631', '514', '48'],
 74: ['261', '327', '11'],
 76: ['536', '658', '40'],
 77: ['820', '310', '33'],
 79: ['791', '257', '36'],
 83: ['612', '501', '34'],
 84: ['580', '454', '34'],
 85: ['462', '562', '36'],
 87: 

Next, convert the values in each list to integer values (at the moment they are strings, which is indicated by the quotes around them).

In [11]:
for item in dict_index_record:
    for i in range(len(dict_index_record[item])):
        dict_index_record[item][i] = int(dict_index_record[item][i])
#         print(dict_index_record[item])
dict_index_record

{2: [890, 312, 41],
 6: [682, 480, 37],
 7: [183, 236, 2],
 9: [744, 429, 42],
 11: [584, 565, 40],
 17: [568, 512, 31],
 24: [516, 555, 31],
 27: [321, 295, 3],
 32: [508, 235, 16],
 33: [351, 250, 4],
 37: [728, 481, 40],
 39: [429, 346, 14],
 41: [460, 655, 38],
 42: [620, 529, 34],
 43: [512, 641, 45],
 45: [472, 600, 34],
 47: [492, 551, 36],
 49: [205, 295, 5],
 50: [230, 210, 7],
 52: [351, 296, 7],
 53: [769, 400, 47],
 57: [315, 337, 8],
 59: [351, 277, 15],
 61: [573, 382, 30],
 64: [555, 566, 36],
 65: [631, 529, 50],
 67: [863, 363, 37],
 69: [161, 314, 3],
 71: [315, 578, 14],
 72: [631, 514, 48],
 74: [261, 327, 11],
 76: [536, 658, 40],
 77: [820, 310, 33],
 79: [791, 257, 36],
 83: [612, 501, 34],
 84: [580, 454, 34],
 85: [462, 562, 36],
 87: [676, 481, 37],
 88: [604, 562, 48],
 89: [465, 615, 32],
 90: [479, 477, 22],
 91: [324, 260, 8],
 92: [334, 417, 14],
 93: [487, 536, 54],
 95: [576, 545, 41],
 97: [360, 278, 6],
 99: [663, 490, 41],
 100: [317, 411, 26],
 101:

Now, we take each record and create a dictionary to house the official win-loss-tie data for each record.

In [12]:
# sample code
lst1 = [4, 7, 2]
lst2 = [3, 6, 5]

outcome = ['win', 'loss', 'tie']
inner_dict = {}
dict_prac = {}
lst = [lst1, lst2]
for thing in range(len(lst)):
    for j in range(len(lst[thing])):
#         print(j)

        inner_dict[outcome[j]] = lst[thing][j]
    dict_prac[str(lst[thing])] = inner_dict
    inner_dict = {}
dict_prac

{'[4, 7, 2]': {'win': 4, 'loss': 7, 'tie': 2},
 '[3, 6, 5]': {'win': 3, 'loss': 6, 'tie': 5}}

The next code block generates a dictionary housing the official win-loss-tie data for each team in a dictionary, `dict_win_loss_tie`.

In [72]:
outcome = ['o_win', 'o_loss', 'o_tie']
inner_dict = {}
dict_official_win_loss_tie = {}

for thing in dict_index_record:
    for j in range(len(dict_index_record[thing])):
#         print(j)

        inner_dict[outcome[j]] = dict_index_record[thing][j]
    dict_official_win_loss_tie[thing] = inner_dict
    inner_dict = {}
dict_official_win_loss_tie

{2: {'o_win': 890, 'o_loss': 312, 'o_tie': 41},
 6: {'o_win': 682, 'o_loss': 480, 'o_tie': 37},
 7: {'o_win': 183, 'o_loss': 236, 'o_tie': 2},
 9: {'o_win': 744, 'o_loss': 429, 'o_tie': 42},
 11: {'o_win': 584, 'o_loss': 565, 'o_tie': 40},
 17: {'o_win': 568, 'o_loss': 512, 'o_tie': 31},
 24: {'o_win': 516, 'o_loss': 555, 'o_tie': 31},
 27: {'o_win': 321, 'o_loss': 295, 'o_tie': 3},
 32: {'o_win': 508, 'o_loss': 235, 'o_tie': 16},
 33: {'o_win': 351, 'o_loss': 250, 'o_tie': 4},
 37: {'o_win': 728, 'o_loss': 481, 'o_tie': 40},
 39: {'o_win': 429, 'o_loss': 346, 'o_tie': 14},
 41: {'o_win': 460, 'o_loss': 655, 'o_tie': 38},
 42: {'o_win': 620, 'o_loss': 529, 'o_tie': 34},
 43: {'o_win': 512, 'o_loss': 641, 'o_tie': 45},
 45: {'o_win': 472, 'o_loss': 600, 'o_tie': 34},
 47: {'o_win': 492, 'o_loss': 551, 'o_tie': 36},
 49: {'o_win': 205, 'o_loss': 295, 'o_tie': 5},
 50: {'o_win': 230, 'o_loss': 210, 'o_tie': 7},
 52: {'o_win': 351, 'o_loss': 296, 'o_tie': 7},
 53: {'o_win': 769, 'o_loss': 

In [73]:
inner_dict

{}

From the above variable, `dict_official_win_loss_tie`, we create feature variables that capture the official `'adjusted'` outcomes.

In [21]:
stuff_to_keep.columns

Index(['Rk', 'School', 'From', 'To', 'Yrs', 'G', 'W', 'L', 'T', 'Pct', 'G.1',
       'W.1', 'L.1', 'T.1', 'Pct.1', 'SRS', 'SOS', 'AP', 'CC', 'Notes'],
      dtype='object')

In [22]:
stuff_to_keep['Pct']

0      0.541
1      0.396
2      0.744
3      0.572
4      0.534
5      0.616
6      0.583
7      0.450
8      0.567
9      0.628
10     0.489
11     0.508
12     0.776
13     0.561
14     0.557
15     0.575
16     0.377
17     0.526
18     0.545
19     0.250
20     0.492
21     0.611
22     0.333
23     0.573
24     0.481
25     0.423
26     0.488
27     0.519
28     0.314
29     0.623
30     0.410
31     0.365
32     0.684
33     0.582
34     0.658
35     0.565
36     0.260
37     0.599
38     0.506
39     0.553
40     0.502
41     0.415
42     0.538
43     0.445
44     0.462
45     0.442
46     0.325
47     0.472
48     0.500
49     0.434
50     0.520
51     0.365
52     0.539
53     0.650
54     0.464
55     0.523
56     0.340
57     0.482
58     0.639
59     0.556
60     0.729
61     0.602
62     0.496
63     0.567
64     0.510
65     0.540
66     0.550
67     0.698
68     0.542
69     0.378
70     0.428
71     0.354
72     0.555
73     0.509
74     0.440
75     0.491
76     0.450

In [70]:
index = [index for index in stuff_to_keep.index]
index
# for item in dict_official_win_loss_tie:
#     print(item)

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129]

In [71]:
# This cell collects all the indices that do not have a difference between the official NCAA win-loss-tie data
# and the initial.
index_to_consider = index
for item in dict_official_win_loss_tie:
    index_to_consider.remove(item)
index_to_consider

[0,
 1,
 3,
 4,
 5,
 8,
 10,
 12,
 13,
 14,
 15,
 16,
 18,
 19,
 20,
 21,
 22,
 23,
 25,
 26,
 28,
 29,
 30,
 31,
 34,
 35,
 36,
 38,
 40,
 44,
 46,
 48,
 51,
 54,
 55,
 56,
 58,
 60,
 62,
 63,
 66,
 68,
 70,
 73,
 75,
 78,
 80,
 81,
 82,
 86,
 94,
 96,
 98,
 103,
 105,
 106,
 108,
 110,
 111,
 112,
 115,
 117,
 118,
 120,
 122,
 125,
 126]

In [75]:
stuff_to_keep.loc[2]['W']

919

In [81]:
initial_outcome = ['W', 'L', 'T']
outcome = ['o_win', 'o_loss', 'o_tie']
inner_dict = {}

for thing in index_to_consider:
    for j in range(len(outcome)):
#         print(j)

        inner_dict[outcome[j]] = stuff_to_keep.loc[thing][initial_outcome[j]]
        dict_official_win_loss_tie[thing] = inner_dict
    inner_dict = {}
dict_official_win_loss_tie

{2: {'o_win': 890, 'o_loss': 312, 'o_tie': 41},
 6: {'o_win': 682, 'o_loss': 480, 'o_tie': 37},
 7: {'o_win': 183, 'o_loss': 236, 'o_tie': 2},
 9: {'o_win': 744, 'o_loss': 429, 'o_tie': 42},
 11: {'o_win': 584, 'o_loss': 565, 'o_tie': 40},
 17: {'o_win': 568, 'o_loss': 512, 'o_tie': 31},
 24: {'o_win': 516, 'o_loss': 555, 'o_tie': 31},
 27: {'o_win': 321, 'o_loss': 295, 'o_tie': 3},
 32: {'o_win': 508, 'o_loss': 235, 'o_tie': 16},
 33: {'o_win': 351, 'o_loss': 250, 'o_tie': 4},
 37: {'o_win': 728, 'o_loss': 481, 'o_tie': 40},
 39: {'o_win': 429, 'o_loss': 346, 'o_tie': 14},
 41: {'o_win': 460, 'o_loss': 655, 'o_tie': 38},
 42: {'o_win': 620, 'o_loss': 529, 'o_tie': 34},
 43: {'o_win': 512, 'o_loss': 641, 'o_tie': 45},
 45: {'o_win': 472, 'o_loss': 600, 'o_tie': 34},
 47: {'o_win': 492, 'o_loss': 551, 'o_tie': 36},
 49: {'o_win': 205, 'o_loss': 295, 'o_tie': 5},
 50: {'o_win': 230, 'o_loss': 210, 'o_tie': 7},
 52: {'o_win': 351, 'o_loss': 296, 'o_tie': 7},
 53: {'o_win': 769, 'o_loss': 

In [97]:
# official win loss tie data for all of the teams

official_wlt = pd.DataFrame(dict_official_win_loss_tie)
official_wlt = official_wlt.T.sort_index()
official_wlt

,o_loss,o_tie,o_win
0,325,12,384
1,230,3,150
2,312,41,890
3,72,5,97
4,422,23,486
5,351,18,568
6,480,37,682
7,236,2,183
8,511,51,678
9,429,42,744


In [79]:
outcome = ['o_win', 'o_loss', 'o_tie']
for j in range(len(outcome)):
    print(j)

0
1
2


In [45]:
lst = [1, 2, 3]
lst

[1, 2, 3]

In [52]:
lst.remove(2)

In [53]:
lst

[1, 3]

In [54]:
for item in lst:
    index.remove(item)
index

[0, 2, 4]